In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def build_efficiency_plots(mean_values):
    fig = plt.figure()
    rt_subplt = fig.add_subplot(212)
    Sp_subplt = fig.add_subplot(221)
    Ep_subplt = fig.add_subplot(222)

    rt_subplt.set_title('runtime')
    rt_subplt.set_xlabel('threads')
    rt_subplt.set_ylabel('time')
    Sp_subplt.set_title('Speedup')
    Sp_subplt.set_xlabel('threads')
    Sp_subplt.set_ylabel("$S_{p}$")
    Ep_subplt.set_title('Efficiency')
    Ep_subplt.set_xlabel('threads')
    Ep_subplt.set_ylabel("$E_{p}$")

    for dimension in sorted(set(mean_values['steps'])):
        sub_df = mean_values.loc[mean_values.steps == dimension]
        one_thread_t = sub_df[sub_df.numThreads == 1]['duration'].astype(float)
        speedup = np.array(one_thread_t) / np.array(sub_df['duration'])
        efficiency = speedup / np.array(sub_df['numThreads'])

        rt_subplt.plot(sub_df['numThreads'], sub_df['duration'],
                       marker="o", label="{steps}".format(steps=dimension))
        Sp_subplt.plot(sub_df['numThreads'], speedup, marker=".",
                       label="{steps}".format(steps=dimension))
        Ep_subplt.plot(sub_df['numThreads'], efficiency,
                       marker=".", label="{steps}".format(steps=dimension))
    rt_subplt.legend()
    plt.subplots_adjust(wspace=1, hspace=1)
    plt.show()

In [3]:
#load data
with open('result.json', 'r') as myfile:
    data=myfile.read()

file_data = json.loads(data)

JSONDecodeError: Expecting ',' delimiter: line 110 column 349 (char 39099)

In [ ]:
# group results by methods
df_by_method = {m:pd.DataFrame([fd[m] for fd in file_data]) for m in file_data[0].keys()}

# set column names & count mean values for each set of params
for df in df_by_method:
    df_by_method[df].columns = ['numThreads','area','steps','error','duration']
    df_by_method[df] = df_by_method[df].groupby(['numThreads', 'steps'], as_index=False).mean()

In [ ]:
df_by_method['Trapezium'].info()#[['numThreads','duration']]

In [ ]:
for df in df_by_method:
    build_efficiency_plots(df_by_method[df])